# Blackjack Simulator: Cheating Dealer Edition 🎲🃏

This Jupyter Notebook is an interactive **Blackjack game simulator** designed to explore probabilities, strategies and the impact of a **weighted (cheating) dealer**.

## Overview
The project simulates rounds of Blackjack between a player and a dealer, implementing standard Blackjack rules:
- Cards 2–10 are valued at their number  
- J, Q, K = 10  
- Ace = 1 or 11 (whichever benefits the hand)

The dealer uses a **weighted deck**, increasing the likelihood of drawing high value cards, giving them an advantage over the player. This allows for analysis of how “cheating” affects game outcomes.

## Features
- Simulate **single or multiple rounds** of Blackjack  
- Track **player and dealer hand totals** and round outcomes  
- Compare results with a **normal dealer vs. cheating dealer**  
- Generate **visualisations**: bar charts, histograms, and cumulative win rates  
- Explore **probabilities, expected values, and game strategies**  

This notebook is a combination of **game simulation, probability analysis, and data visualisation**, demonstrating Python programming and interactive notebook design.


In [1]:
import random
import matplotlib
import pandas

In [2]:
deck = {
    '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10,
    'J': 10, 'Q': 10, 'K': 10, 'A': 11
}

In [3]:
# Draw cards for player and dealer
playerHand = random.choices(
    population=list(deck.keys()),
    weights=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    k=2
)

dealerHand = random.choices(
    population=list(deck.keys()),
    weights=[1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2],
    k=2
)
print("Dealer Hand:", dealerHand[0], "?")
print("Player Hand:",playerHand)
print("Total:", sum([deck[card] for card in playerHand]))


Dealer Hand: K ?
Player Hand: ['4', 'J']
Total: 14


In [ ]:
# Single round simulation
while True:
    choice = input("Do you want to hit or stand? (h/s): ").lower()
    if choice == 'h':
        card = random.choices(
    population=list(deck.keys()),
    weights=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    k=1)
        playerHand.append(card[0])
        total = sum([deck[card] for card in playerHand])
        print("Player Hand:", playerHand)
        print("Total:", total)
        if total > 21:
            print("Bust! You lose.")
            exit()
    elif choice == 's':
        break


# Dealer's turn
print("Dealer's turn...")
print("Dealer Hand:", dealerHand)
while sum([deck[card] for card in dealerHand]) < 17:
    card = random.choices(
        population=list(deck.keys()),
        weights=[1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2],
        k=1
    )
    dealerHand.append(card[0])
    print("Dealer Hand:", dealerHand)
dealerTotal = sum([deck[card] for card in dealerHand])
playerTotal = sum([deck[card] for card in playerHand])
print("Dealer Total:", dealerTotal)
print("Player Total:", playerTotal)
# Determine winner
if dealerTotal > 21 or playerTotal >= dealerTotal:
    print("You win!")
elif playerTotal < dealerTotal:
    print("Dealer wins!")

Player Hand: ['4', 'J', '4']
Total: 18
Dealer's turn...
Dealer Hand: ['K', '2']
Dealer Hand: ['K', '2', 'K']
Dealer Total: 22
Player Total: 18
You win!
